### libs

In [163]:
import dspy
import openai
from openai import OpenAI
import pandas as pd
import os
import json
import hashlib
from tqdm import tqdm
import chromadb
import weaviate
from dspy.retrieve.weaviate_rm import WeaviateRM

import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

import time

# SECRET KEY OPENAI
secretk = os.getenv("OPENAI_API_KEY")
#client = OpenAI(api_key=secretk)
openai.api_key = secretk
client = OpenAI(api_key=secretk)

import pandas as pd
import os

os.chdir(os.path.abspath(os.curdir))

In [164]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'app_okrs_streamlit.py',
 'apresentacao_GP_vf_OK.pdf',
 'apresentacao_GP_vf_OK.pptx',
 'arquivo_FC.csv',
 'artifacts_consulting.py',
 'assertion.log',
 'azure_openai_usage.log',
 'builder_image_docker.py',
 'docker-compose.yml',
 'dockerfile',
 'docker_images',
 'ERROS E SOLUCOES DSPY, WEAVIATE, ML.txt',
 'gestaoprojetos',
 'inference_scoring.py',
 'local_cache',
 'mapa_indicadores.xlsx',
 'mapa_indicadores_GPT.csv',
 'mapa_indicadores_GPT.xlsx',
 'mapa_indicadores_GPT_.csv',
 'mapa_indicadores_GPT_okrs.csv',
 'mapa_indicadores_GPT_okrs.json',
 'mapa_indicadores_RAG_21062024.xlsx',
 'meu_projeto_ml',
 'mlflow.db',
 'mlruns',
 'modeloOkrs.py',
 'modeloOkrs_II.py',
 'modelo_dspy_okr',
 'modelo_dspy_okrs',
 'modelo_oportune_12062024.json',
 'modelo_oportune_12062024_okr.json',
 'modelo_oportune_12062024_okr.pkl',
 'modelo_oportune_13062024_okr.pkl',
 'modelo_oportune_17062024_okr.pkl',
 'modelo_oportune_21062024_okr.pkl',
 'modelo_oportune_2

In [165]:
import re

# Função para limpar texto
def clean_text(text):
    # Decodificar caracteres Unicode
    text = text.encode().decode('utf-8')
    # Remover caracteres indesejados específicos (exemplo: \n, \r, etc.)
    text = re.sub(r'[\n\r]', ' ', text)
    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [6]:
df = pd.read_excel('OportunidadesMelhoriaJuntada.xlsx', header=3)

In [7]:
df_ = df.iloc[1:, 2:9]
df_

,Frente,PROCESSO,ETAPA RELACIONADA,IDEIA,TIPO DE MELHORIA,DESCONEXÕES \n(EVENTO/CAUSA/CONSEQUÊNCIA),OPORTUNIDADES DE MELHORIA \n(SOLUÇÃO / INVESTIMENTO / GANHOS)
1,Ambiental,1. Realizar Admissão de Colaborador,Receber documentação do candidato,\nEstruturar projeto de expansão para a impla...,Sistemas / Infraestrutura,Evento: Algumas cidades do negócio SAS (Osasco...,Solução: Criar projeto de expansão da implanta...
2,Ambiental,1. Realizar Admissão de Colaborador,Alterar status do candidato | Entregar ficha d...,Parametrizar sistema Yube com mecanismos de n...,Sistemas / Infraestrutura,Evento: Não é enviado notificação para o DP so...,Solução: Parametrizar a plataforma de admissão...
3,Ambiental,1. Realizar Admissão de Colaborador,Imprimir documentação do colaborador | Entreg...,Analisar junto ao fornecedor da plataforma Yub...,Sistemas / Infraestrutura,Evento: Atualmente os documentos não são armaz...,Solução: Analisar junto ao fornecedor da plata...
4,Ambiental,1. Realizar Admissão de Colaborador,Conferir documentação,Realizar análise/entendimento junto ao Yube pa...,Sistemas / Infraestrutura,Evento: O setor de Departamento pessoal realiz...,Solução: Realizar análise/entendimento junto a...
5,Ambiental,1. Realizar Admissão de Colaborador,Conferir documentação | Cadastrar benefícios d...,Definir papéis e responsabilidades das ativida...,Fluxo de trabalho / Regras de negócio,Evento: Algumas responsabilidades das atividad...,Solução: Revisar processo para definir e padro...
...,...,...,...,...,...,...,...
154,Infraestrutura,Gerir matérias e Serviços de Obra - Combustível,Necessidade de compra,Desenvolver um painel interativo para o acompa...,Fluxo de trabalho / Regras de negócio,Evento: Atualmente a gestão de estoque de dies...,Solução: Desenvolver um painel interativo para...
155,Infraestrutura,Gerir materiais de obras de infraestrutura,Analisar necessidade de entrega de material,Desenvolver uma solução para monitorar e sinal...,Sistemas / Infraestrutura,Evento: Atualmente não é sinalizado quando há ...,\nSolução: Realizar desenvolvimento de uma so...
156,Infraestrutura,Gerir materiais de obras de infraestrutura,Realizar entrega de materias,"Digitalizar o controle de entrega de EPI, gelo...",Fluxo de trabalho / Regras de negócio,Evento: Necessidade de formalização de process...,\nSolução: Digitalizar o processo de controle...
157,Infraestrutura,Gerir materiais de obras de infraestrutura,Registrar recebimento de insumos/Realizar a e...,Automatizar o apontamento de liberações de mat...,Sistemas / Infraestrutura,Evento: Atualmente o apontamento de liberação ...,Solução: Automatizar o apontamento de liberaçõ...


In [8]:
df_.columns

Index(['Frente', 'PROCESSO', 'ETAPA RELACIONADA', 'IDEIA', 'TIPO DE MELHORIA',
       'DESCONEXÕES \n(EVENTO/CAUSA/CONSEQUÊNCIA)',
       'OPORTUNIDADES DE MELHORIA \n(SOLUÇÃO / INVESTIMENTO / GANHOS)'],
      dtype='object')

In [9]:
colunas_nm = {
       'Frente':'frente', 'PROCESSO':'processo', 'ETAPA RELACIONADA':'etapa_relacionada', 'IDEIA':'ideia', 
       'TIPO DE MELHORIA':'tipo_de_melhoria', 'DESCONEXÕES \n(EVENTO/CAUSA/CONSEQUÊNCIA)':'desconexoes',
       'OPORTUNIDADES DE MELHORIA \n(SOLUÇÃO / INVESTIMENTO / GANHOS)' : 'oportunidades'
}

In [10]:
df_.rename(columns=colunas_nm, inplace=True)

In [11]:
df_[:5]

,frente,processo,etapa_relacionada,ideia,tipo_de_melhoria,desconexoes,oportunidades
1,Ambiental,1. Realizar Admissão de Colaborador,Receber documentação do candidato,\nEstruturar projeto de expansão para a impla...,Sistemas / Infraestrutura,Evento: Algumas cidades do negócio SAS (Osasco...,Solução: Criar projeto de expansão da implanta...
2,Ambiental,1. Realizar Admissão de Colaborador,Alterar status do candidato | Entregar ficha d...,Parametrizar sistema Yube com mecanismos de n...,Sistemas / Infraestrutura,Evento: Não é enviado notificação para o DP so...,Solução: Parametrizar a plataforma de admissão...
3,Ambiental,1. Realizar Admissão de Colaborador,Imprimir documentação do colaborador | Entreg...,Analisar junto ao fornecedor da plataforma Yub...,Sistemas / Infraestrutura,Evento: Atualmente os documentos não são armaz...,Solução: Analisar junto ao fornecedor da plata...
4,Ambiental,1. Realizar Admissão de Colaborador,Conferir documentação,Realizar análise/entendimento junto ao Yube pa...,Sistemas / Infraestrutura,Evento: O setor de Departamento pessoal realiz...,Solução: Realizar análise/entendimento junto a...
5,Ambiental,1. Realizar Admissão de Colaborador,Conferir documentação | Cadastrar benefícios d...,Definir papéis e responsabilidades das ativida...,Fluxo de trabalho / Regras de negócio,Evento: Algumas responsabilidades das atividad...,Solução: Revisar processo para definir e padro...


In [167]:
weaviate_client = weaviate.connect_to_local(
                        headers = {
                                    "X-OpenAI-Api-Key": secretk
                        }
)

#schema_ = weaviate_client.schema.get()

# deletando a classe existente
#weaviate_client.schema.delete_all()
#print(f"Classe limpa: {weaviate_client.schema.get()}, está ativa -> {weaviate_client.is_ready()}")

In [168]:
df_.columns

Index(['frente', 'processo', 'etapa_relacionada', 'ideia', 'tipo_de_melhoria',
       'desconexoes', 'oportunidades'],
      dtype='object')

#### Classe

In [169]:
try:
    weaviate_client.connect()
    weaviate_client.collections.delete("DocsOportunidades")
except Exception as e:
    print(f"Erro ao deletar classe! {e}")
finally:
    print("Dados deletados da classe.")
    weaviate_client.close()


Dados deletados da classe.


In [170]:

try:
    weaviate_client.connect()
    minha_classe = {
        
        "class" : "DocsOportunidades", 
        "description":"Aqui seguem anotações de como os processos e problemas estão sendo resolvidos pela consultoria de Gestão de Projetos e Processos.",
        "invertedIndexConfig": {
            "cleanupIntervalSeconds": 60
          },
                    "properties": [
                            {"name": "oportunidades",
                             "dataType": ["text"],
                             "description":"Contém o texto base do documento com OPORTUNIDADES DE MELHORIA(SOLUÇÃO INVESTIMENTO GANHOS)", 
                            }, 
                        
                            {"name": "desconexoes",
                             "dataType": ["text"],
                             "description":"Contém o texto base do documento com DESCONEXÕES(EVENTO CAUSA CONSEQUÊNCIA).", 
                            }, 
    
                            {"name": "tipo_de_melhoria",
                             "dataType": ["string"],
                             "description":"Tipos de melhorias."
                             },
                        
                            {"name": "ideia",
                             "dataType": ["string"],
                             "description":"Quais as ideias que foram tidas para resolver o problema ou problemas."
                             }, 
                        
                             {"name": "etapa_relacionada",
                             "dataType": ["string"],
                             "description":"Etapas do processo relacionadas no momento."
                             },
                            {"name": "processo",
                                 "dataType": ["string"],
                                 "description":"tipo de processo que esta envolvido."
                                 },
                            {"name": "frente",
                                 "dataType": ["string"],
                                 "description":"Qual a frente da empresa esta sendo atendida."
                                 },
                      ],
        
                "vectorizer": "text2vec-openai",
                
                # gpt-3.5-turbo | gpt-3.5-turbo-16k | gpt-3.5-turbo-1106 | gpt-4 | gpt-4-32k | gpt-4-1106-preview
                
                "moduleConfig": {
                    "generative-openai": {
                      "model": "gpt-3.5-turbo-16k",
                      "temperatureProperty": 0.7,  
                      "maxTokensProperty":500,  
                      #"frequencyPenaltyProperty": <frequency_penalty>,  // Optional, applicable to both OpenAI and Azure OpenAI
                      #"presencePenaltyProperty": <presence_penalty>,  // Optional, applicable to both OpenAI and Azure OpenAI
                      "topPProperty": 3,
                    },
                    
                    "text2vec-openai": {
                      "model": "text-embedding-3-large",
                      "dimensions": 3072,
                      "type": "text"          
                    }
                    
                  },
                
                # Configure the vector index
                "vectorIndexType": "hnsw",
                "vectorIndexConfig": {
                    "distance": "cosine",
                    "pq": {
                        "enabled": True,
                        "segments": 192
                    },
                },
                # Configure the inverted index
                "indexTimestamps": True,
                "indexNullState": True,
                "indexPropertyLength": True,
            
            }

    weaviate_client.collections.create_from_dict(minha_classe)
    print("Nova classe criada!")   

except Exception as e:
    print(f"ERRO: {e}")
finally:
    weaviate_client.close()
    print("Tudo certo! \n(client closed)")

Nova classe criada!
Tudo certo! 
(client closed)


### Tratamento de dados e BATCH ADD

In [171]:
df_.columns

Index(['frente', 'processo', 'etapa_relacionada', 'ideia', 'tipo_de_melhoria',
       'desconexoes', 'oportunidades'],
      dtype='object')

In [172]:
df_.reset_index(drop=True, level=0, inplace=True)

In [173]:
df_['ideia'].fillna("Sem preenchimento", inplace=True)

C:\Users\ThiagoBluhm\AppData\Local\Temp\ipykernel_29848\155809325.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_['ideia'].fillna("Sem preenchimento", inplace=True)


In [174]:
df_.isna().value_counts()

frente  processo  etapa_relacionada  ideia  tipo_de_melhoria  desconexoes  oportunidades
False   False     False              False  False             False        False            158
Name: count, dtype: int64

In [175]:
lista = [] 
for i in range(len(df_)):
    lista.append({
                    'frente': df_["frente"][i],
                    'processo': df_["processo"][i],
                    'etapa_relacionada': df_["etapa_relacionada"][i], 
                    'ideia': df_["ideia"][i], 
                    'tipo_de_melhoria': df_["tipo_de_melhoria"][i],
                    'desconexoes':df_["desconexoes"][i], 
                    'oportunidades' : df_["oportunidades"][i]
                 })
    
lista

[{'frente': 'Ambiental',
  'processo': '1. Realizar Admissão de Colaborador',
  'etapa_relacionada': 'Receber documentação do candidato',
  'ideia': '\nEstruturar projeto de expansão para a  implantação da plataforma de admissão digital em todas as filias e negócios do grupo, para padronizar o recebimento de documentação digital de forma corporativa',
  'tipo_de_melhoria': 'Sistemas / Infraestrutura',
  'desconexoes': 'Evento: Algumas cidades do negócio SAS (Osasco, Porto Velho, Manaus e Taubaté) e demais negócios do grupo recebem a documentação do candidato de forma manual.  O piloto de admissão digital está em andamento na ECOFOR , incorporação e MGC.\nCausa: As cidades não possuem o sistema/ não foram capacitados para o uso da ferramenta como meio de recebimento da documentação.\nConsequência: Trabalho manual, demora no processo, espaço de armazenamento físico comprometido, risco de perda ou dano da documentação, dificuldade na análise de dados pois impacto na análise de dados pois 

In [176]:
treino = [dspy.Example(question=f"""{df_['desconexoes'][i].split("Evento:")[1]}""", 
                       answer=df_.loc[i, "oportunidades"]).with_inputs("question") for i in range(len(df_))]

In [177]:
import requests
import json

try:
    weaviate_client.connect()
    with weaviate_client.batch.dynamic() as batch:  # Initialize a batch process
        for i, d in enumerate(lista):  # Batch import data
            try:
                properties = {
                        'frente': d["frente"],
                        'processo': d["processo"],
                        'etapa_relacionada': d["etapa_relacionada"], 
                        'ideia': d["ideia"], 
                        'tipo_de_melhoria': d["tipo_de_melhoria"],
                        'desconexoes':d["desconexoes"], 
                        'oportunidades' : d["oportunidades"]
        
                    }
            except Exception as e:
                print(f"ERRO: {e}")
                
            try:
                batch.add_object(
                    properties=properties,
                    collection="DocsOportunidades"
                )
                print(f"importing question: {i + 1}")
                              
            except Exception as e:
                print(f'Erro ao fazer add_object --> {e}')

except Exception as e:
    print(f"ERRO: {e} - INICIO")

finally:
    weaviate_client.close()

importing question: 1
importing question: 2
importing question: 3
importing question: 4
importing question: 5
importing question: 6
importing question: 7
importing question: 8
importing question: 9
importing question: 10
importing question: 11
importing question: 12
importing question: 13
importing question: 14
importing question: 15
importing question: 16
importing question: 17
importing question: 18
importing question: 19
importing question: 20
importing question: 21
importing question: 22
importing question: 23
importing question: 24
importing question: 25
importing question: 26
importing question: 27
importing question: 28
importing question: 29
importing question: 30
importing question: 31
importing question: 32
importing question: 33
importing question: 34
importing question: 35
importing question: 36
importing question: 37
importing question: 38
importing question: 39
importing question: 40
importing question: 41
importing question: 42
importing question: 43
importing question: 

In [178]:
df_.columns

Index(['frente', 'processo', 'etapa_relacionada', 'ideia', 'tipo_de_melhoria',
       'desconexoes', 'oportunidades'],
      dtype='object')

In [111]:
s = f"""Sugestao de melhoria para pagamentos das empresas que utilizam 
um sistema que nao registra lancamento financeiro automatico?
"""
response = (
   weaviate_client.query
  .get("DocsOportunidades", ["oportunidades"])
  .with_generate(grouped_task=s, grouped_properties=["ideia", "etapa_relacionada"] )
  .with_near_text({
    "concepts": ["prazos"]
  })
  .with_limit(2)
).do()

print(clean_text(json.dumps(response, indent=2)))

C:\Users\ThiagoBluhm\anaconda3\envs\NLP_env\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
C:\Users\ThiagoBluhm\anaconda3\envs\NLP_env\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


AttributeError: 'WeaviateClient' object has no attribute 'query'

In [77]:
s = """Como melhorar prazos na empresa se ela pratica prazos diferentes em cada sede?
"""
response = (
        weaviate_client.query.get("Documentos", ['oportunidades'])
        .with_hybrid(
            query=s,
            properties=['frente', 'processo', 'etapa_relacionada', 'ideia', 'tipo_de_melhoria', 
                        'desconexoes', 'oportunidades'],
            alpha=0.5
        )
        .with_limit(3)
        .with_additional(["score", "explainScore", "distance"])
        .do()
    )


#print(f"Original: {s}\n")
for i in range(len(response['data']['Get']['Documentos'])):
    texto = json.dumps(response['data']['Get']['Documentos'][0]["oportunidades"], indent=2)
    print(f'{clean_text(texto)} \n')

"Solu\u00e7\u00e3o: Realizar revis\u00e3o dos SLAS e atividades do processo de Admiss\u00e3o relacionados a Gest\u00e3o de Pessoas e Departamento de Pessoal para padroniza\u00e7\u00e3o de forma corporativa.\nInvestimento: N/A\nGanhos: Promover a efici\u00eancia operacional, melhorar a comunica\u00e7\u00e3o interna e garantir a padroniza\u00e7\u00e3o dos prazos e consist\u00eancia da qualidade do processo." 

"Solu\u00e7\u00e3o: Realizar revis\u00e3o dos SLAS e atividades do processo de Admiss\u00e3o relacionados a Gest\u00e3o de Pessoas e Departamento de Pessoal para padroniza\u00e7\u00e3o de forma corporativa.\nInvestimento: N/A\nGanhos: Promover a efici\u00eancia operacional, melhorar a comunica\u00e7\u00e3o interna e garantir a padroniza\u00e7\u00e3o dos prazos e consist\u00eancia da qualidade do processo." 

"Solu\u00e7\u00e3o: Realizar revis\u00e3o dos SLAS e atividades do processo de Admiss\u00e3o relacionados a Gest\u00e3o de Pessoas e Departamento de Pessoal para padroniza\u00e

### DSPy

In [179]:
_params4o = {"lm" : dspy.OpenAI(model='gpt-4o', max_tokens=2048, temperature=0.2), 
             "rm" : WeaviateRM("DocsOportunidades", weaviate_client = weaviate_client, 
                             # PASSAMOS O COLLECTION TEXT KEY QUE SERA USADO PARA CLASSIFICAR O TEXTO => (CONTEXT)
                             weaviate_collection_text_key="oportunidades"
                            )
           }

# Passando kwargs
# avaliar as configs usando dspy.settings.config...
dspy.settings.configure(**_params4o)
weaviate_client.connect()

### DSPy Oportune

In [180]:
class Oportune(dspy.Signature):
    
    """
       ** Contexto: **
       Você é um experiente Gerente de Projetos e Processos de uma grande empresa que presta consultoria na área de projetos e possui
       diversas certificações internacionais. 
       Sua tarefa principal é mapear os problemas que são identificados na consultoria, perceber quais são as oportunidades de melhoria
       que irão favorecer o cliente e o projeto como um todo e por fim ter um plano de tarefas bem definido, em tópicos, para o problema
       relatado e identificado juntamente com o cliente. Sua idéias para melhorar os processos da empresa com oportunidades aprendidas 
       durante o seu exercício de consultoria em outros clientes fará toda diferença.
       
    """        
  
    # TEXTO DO PROBLEMA DE ENTRADA
    question = dspy.InputField()
    # AQUI TEMOS AS OPORTUNIDADES QUE VEM DA BASE FRIA (NO CASO ESTAMOS USANDO 5 PASSAGES)
    context = dspy.InputField(desc="""traga as oportunidades mais relevantes para o cenário do cliente em questão.""")    
   
    # RESPOSTA SEGUINDO OS FEWSHOTS dspy.Example()
    answer = dspy.OutputField(desc="""Dê sua resposta com as 5 melhores oportunidades, cite as possíveis soluções para cada solução, 
                                      cada uma com o investimento que irá ser necessário e os ganhos que irá trazer a empresa, 
                                      modelo de resposta:
                                      
                                      **Oportunidade de Melhoria** : Automatização de Captura de Informações de Pagamentos:
                                      
                                      **Solução** : Desenvolver uma integração com o sistema bancário para captura automática de informações 
                                      sobre pagamentos realizados e recusados. Isso inclui a baixa automática dos pagamentos no sistema e 
                                      notificações para as áreas interessadas em caso de problemas.
                                      
                                      **Investimento** : Horas para desenvolvimento de customizações e implantação do projeto.
                                      
                                      **Ganhos:** Automação de atividades manuais e repetitivas, maior agilidade no processo, redução do 
                                      risco de erros operacionais e simplificação da disseminação de informações entre as partes interessadas. 
                                      
                                      Dê sugestões simples e eficientes de acordo com o seu conhecimento e contexto. 
                                      Pense passo a passo, gerando mais de uma resposta, avalie cada passo e veja qual dos sugeridos 
                                      é o mais adequado. 
                                      RESPONDA em Português do Brasil.
                            """)

class OportuneRAG(dspy.Module):
    def __init__(self, num_passages = 5): 
        super().__init__()
       
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(Oportune) # Step by Step Cadeia de pensamento... ToT, BoT...

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)        
        return dspy.Prediction(context=context, answer=prediction.answer)

In [135]:
# escreve pouco e recebe muito...

In [181]:
from dspy.teleprompt import LabeledFewShot

labeled_fewshot_optimizer = LabeledFewShot(k=16)
modeloCompilado = labeled_fewshot_optimizer.compile(student = OportuneRAG(), trainset=treino)

In [185]:
#modeloCompilado.save('modelo_oportune_05112024.json')

In [186]:
s = f"""Empresa do ramo de construção, precisa melhorar seu processo de suprimentos, visto que o valor gasto é maior que 
foi orçado para a área. Verificado que a ordem de compra não é vinculado ao orçamento
"""
caminho = r"modelo_oportune_05112024.json"
modelo = OportuneRAG()
modelo.load(path=caminho)
resposta = modelo(question = s)

In [187]:
print(resposta.answer)

**Oportunidade de Melhoria**: Vinculação da Ordem de Compra ao Orçamento  
**Solução**: Implementar um sistema de integração entre as ordens de compra e o orçamento, garantindo que todas as compras sejam aprovadas dentro dos limites orçamentários estabelecidos.  
**Investimento**: Horas de TI para desenvolvimento e integração do sistema.  
**Ganhos**: Controle mais rigoroso dos gastos, evitando despesas não planejadas e garantindo que as compras estejam alinhadas com o orçamento aprovado.

**Oportunidade de Melhoria**: Painel de Administração de Compras  
**Solução**: Criar um painel para administrar a compra de insumos para o portfólio de todas as obras, permitindo uma visão consolidada e estratégica das necessidades de suprimentos.  
**Investimento**: Horas da TI e da equipe técnica para desenvolvimento e homologação do painel.  
**Ganhos**: Maior poder de barganha nas negociações de insumos genéricos, redução de retrabalho e risco de erros, processo de compras mais eficiente.

**Opo

In [158]:
print(_params4o['lm'].inspect_history(n=10))




Você é um analista de projetos e processos de uma grande empresa de consultoria. 
Sua tarefa é oferecer idéias para melhorar os processos da empresa com oportunidades aprendidas 
durante o seu exercício de consultoria em outros clientes.

---

Question: Há necessidade da emissão de relatório no Portal RM para poder visualizar quais os ASO's próximos a vencer. Causa: Sistema não gera alerta de vencimento dos periódicos Consequência: Comunicação tardia e gerenciamento manual.
Answer: Solução: Realizar a sinalização via sistema para o SESMT e aos colaboradores dos ASO's próximos ao vencimento, garantindo a execução do ASO na periodicidade prevista e definir barreiras, exemplo: bloqueios nos acessos, para evitar a não realização do ASO. Investimento: Implantação de automação no sistema. Ganhos: Redução do risco da comunicação tardia sobre o vencimento do ASO e de trabalho manual.

Question: Os certificado de aprovação dos EPI's são arquivados na pasta do SharePoint. Causa: Controle do c

In [132]:
#print("""Context: \n1. **Integração de Sistemas**: Desenvolver uma integração entre o sistema de pagamentos e o sistema financeiro para que os lançamentos sejam registrados automaticamente.\n2. **Automação de Relatórios**: Implementar uma ferramenta de automação que gere relatórios financeiros automaticamente com base nos dados de pagamento.\n3. **Padronização de Processos**: Padronizar os procedimentos de pagamento, incluindo a criação de templates e checklists.\n4. **Implementação de SLA's (Service Level Agreements)**: Estabelecer acordos de nível de serviço para garantir que os pagamentos sejam processados dentro de um prazo específico.\n5. **Comunicação e Treinamento**: Melhorar a comunicação entre as áreas e oferecer treinamento adequado para a equipe.\n\nQuestion: Fazer algumas sugestões de melhoria para pagamentos das empresas que utilizam um sistema que não registra lançamento financeiro automático?\n\nReasoning: Vamos pensar passo a passo para produzir a resposta. Vamos considerar as causas e consequências de não ter um registro financeiro automático e como podemos melhorar esse processo.\n\n1. **Identificação das Causas**:\n   - Falta de integração entre sistemas financeiros e o sistema de pagamentos.\n   - Processos manuais que consomem tempo e são suscetíveis a erros.\n   - Falta de padronização nos procedimentos de pagamento.\n\n2. **Análise das Consequências**:\n   - Atrasos nos pagamentos devido ao tempo necessário para processar manualmente cada transação.\n   - Maior risco de erros humanos, como duplicidade de pagamentos ou omissões.\n   - Dificuldade em manter uma base de dados atualizada e precisa.\n\n3. **Propostas de Soluções**:\n   - **Integração de Sistemas**: Desenvolver uma integração entre o sistema de pagamentos e o sistema financeiro para que os lançamentos sejam registrados automaticamente. Isso pode incluir a captura automática de informações de pagamento e a atualização das bases de dados.\n     - **Investimento**: Horas de desenvolvimento e customização.\n     - **Ganhos**: Automação de processos manuais, redução de erros, maior agilidade e confiabilidade no processamento de pagamentos.\n\n   - **Automação de Relatórios**: Implementar uma ferramenta de automação que gere relatórios financeiros automaticamente com base nos dados de pagamento. Isso pode incluir a geração de relatórios diários, semanais ou mensais que detalhem todos os pagamentos realizados.\n     - **Investimento**:""")

In [133]:
s = f"""Sugestao de melhoria para pagamentos das empresas que utilizam 
um sistema que nao registra lancamento financeiro automatico?
"""
resposta = modeloCompilado(question = s)
print(resposta)

Prediction(
    context=['Solução: Analisar os principais tipos e as principais causas dos pagamentos remotos, identificar oportunidades de renegociação dos pagamentos remotos e implementar melhorias, visando reduzir o volume desse tipo de pagamento. Atualizar política de pagamentos com as novas regras definidas e comunicar fornecedores.\nInvestimento:  N/A\nGanhos: Reduzir a complexidade da análise no fechamento dos lotes de pagamentos, reduzindo o retrabalho, o tempo de fechamento dos lotes e o risco de erros. Melhorar a eficiência do processo e o uso de recursos.', 'Solução: Desenvolver uma integração com o sistema bancário que permita a captura automática de informações referentes aos pagamentos realizados com sucesso, executando a baixa automática desses pagamentos no sistema e atualizando as bases de dados/ relatórios de gestão de atividades. \nInvestimento: Horas para desenvolvimento de customizações e implantação do projeto. \nGanhos: Automação de atividades manuais e repetitiv

In [ ]:
clean_text("""
1. **Automatização de Captura de Informações de Pagamentos:**\n   - **Solução:** Desenvolver uma integração com o 
sistema bancário para captura automática de informações sobre pagamentos realizados e recusados. Isso inclui a baixa 
automática dos pagamentos no sistema e notificações para as áreas interessadas em caso de problemas.\n   - 
**Investimento:** Horas para desenvolvimento de customizações e implantação do projeto.\n   - **Ganhos:** 
Automação de atividades manuais e repetitivas, maior agilidade no processo, redução do risco de erros operacionais 
e simplificação da disseminação de informações entre as partes interessadas.\n\n2. **Padronização de Dados e 
Processos:**\n   - **Solução:** Padronizar os dados e telas de validação dos pagamentos lançados no sistema, 
simplificando a aprovação dos lançamentos e a conferência dos lotes de pagamentos. Implementar campos padronizados 
como "AP - apto a pagar" e "não pertencente ao lote" para eliminar a necessidade de exclusão manual de notas fiscais.\n   - 
**Investimento:** Horas para desenvolvimento de customizações e treinamento das equipes envolvidas.\n   - **Ganhos:** 
Simplificação da validação dos pagamentos, eliminação de retrabalho, redução da sobrecarga do time, mitigação de riscos 
de atrasos e pagamentos de multas e juros.\n\n3. **Renegociação de Pagamentos Remotos:**\n   - **Solução:** Analisar os 
tipos e causas dos pagamentos remotos, identificar oportunidades de renegociação e implementar melhorias para reduzir o 
volume desses pagamentos. Atualizar a política de pagamentos com as novas regras e comunicar fornecedores.\n   - 
**Investimento:** N/A\n   - **Ganhos:** Redução da complexidade na análise e fechamento dos lotes de pagamentos, 
redução do retrabalho, tempo de fechamento dos lotes e risco de erros, além de melhorar a eficiência do processo e o 
uso de recursos.\n\n4. **Centralização e Confiabilidade das Informações:**\n   - **Solução:** Desenvolver uma integração 
com o sistema bancário para capturar automaticamente informações atualizadas sobre saldos e movimentações das contas. 
Executar um relatório BI com essas informações de forma organizada e com indicadores visuais simples.\n   - 
**Investimento:** Horas para desenvolvimento de customizações e implantação do projeto.\n   - 
**Ganhos:** Automação de atividades manuais e repetitivas, redução dos retrabalhos e dos riscos de erros relacionados 
às análises e coleta dos dados, maior agilidade e transparência no processo.\n\n5. **Comunicação e Notificações Eficientes:**
\n   - **Solução:** Implementar um sistema de notificações automáticas para comunicar as áreas interessadas 
(financeiro e solicitante) sobre o status dos pagamentos e ações necessárias para resolver problemas.\n   - **Investimento:** 
Horas para desenvolvimento de customizações e implantação do projeto.\n   - **Ganhos:** Maior agilidade no processo, 
simplificação da disseminação de informações, redução do risco de erros operacionais e melhoria na comunicação interna.
\n\nAo implementar essas sugestões, a empresa poderá melhorar significativamente a eficiência dos seus processos de pagamento, 
reduzir o risco de erros e retrabalhos, e aumentar a agilidade e transparência das operações 
financeiras.
""")

In [ ]:
PERGUNTA = f"""Sugestao de melhoria para pagamentos das empresas que utilizam 
um sistema que nao registra lancamento financeiro automatico?
"""

RESPOSTA2: """
1. **Automatização de Captura de Informações de Pagamentos:** - **Solução:** Desenvolver uma integração com o sistema 
bancário para captura automática de informações sobre pagamentos realizados e recusados. Isso inclui a baixa automática 
dos pagamentos no sistema e notificações para as áreas interessadas em caso de problemas. - **Investimento:** Horas para 
desenvolvimento de customizações e implantação do projeto. - **Ganhos:** Automação de atividades manuais e repetitivas, 
maior agilidade no processo, redução do risco de erros operacionais e simplificação da disseminação de informações entre 
as partes interessadas. 

2. **Padronização de Dados e Processos:** - **Solução:** Padronizar os dados e telas de validação 
dos pagamentos lançados no sistema, simplificando a aprovação dos lançamentos e a conferência dos lotes de pagamentos. 
Implementar campos padronizados como "AP - apto a pagar" e "não pertencente ao lote" para eliminar a necessidade de 
exclusão manual de notas fiscais. - **Investimento:** Horas para desenvolvimento de customizações e treinamento das 
equipes envolvidas. - **Ganhos:** Simplificação da validação dos pagamentos, eliminação de retrabalho, redução da sobrecarga 
do time, mitigação de riscos de atrasos e pagamentos de multas e juros. 

3. **Renegociação de Pagamentos Remotos:** - 
**Solução:** Analisar os tipos e causas dos pagamentos remotos, identificar oportunidades de renegociação e implementar 
melhorias para reduzir o volume desses pagamentos. Atualizar a política de pagamentos com as novas regras e comunicar 
fornecedores. - **Investimento:** N/A - **Ganhos:** Redução da complexidade na análise e fechamento dos lotes de pagamentos, 
redução do retrabalho, tempo de fechamento dos lotes e risco de erros, além de melhorar a eficiência do processo e o uso de 
recursos. 

4. **Centralização e Confiabilidade das Informações:** - **Solução:** Desenvolver uma integração com o sistema 
bancário para capturar automaticamente informações atualizadas sobre saldos e movimentações das contas. Executar um relatório 
BI com essas informações de forma organizada e com indicadores visuais simples. - **Investimento:** Horas para 
desenvolvimento de customizações e implantação do projeto. - **Ganhos:** Automação de atividades manuais e repetitivas, 
redução dos retrabalhos e dos riscos de erros relacionados às análises e coleta dos dados, maior agilidade e transparência 
no processo. 

5. **Comunicação e Notificações Eficientes:** - **Solução:** Implementar um sistema de notificações automáticas 
para comunicar as áreas interessadas (financeiro e solicitante) sobre o status dos pagamentos e ações necessárias para 
resolver problemas. - **Investimento:** Horas para desenvolvimento de customizações e implantação do projeto. - 
**Ganhos:** Maior agilidade no processo, simplificação da disseminação de informações, redução do risco de erros operacionais 
e melhoria na comunicação interna. Ao implementar essas sugestões, a empresa poderá melhorar significativamente a eficiência 
dos seus processos de pagamento, reduzir o risco de erros e retrabalhos, e aumentar a agilidade e transparência das operações 
financeiras.
"""

In [ ]:
_params['lm'].inspect_history(n=1)